### **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from configs.paths import abfss_path, checkpoint_path


In [0]:
df = spark.read.format("parquet")\
    .load(abfss_path("bronze", "orders"))


In [0]:
df = df.withColumnRenamed("_rescued_data", "rescued_data")
df = df.drop("rescued_data")

In [0]:

df = df.withColumn("order_date", to_timestamp(col('order_date')))
df = df.withColumn("year", year(col("order_date")))

In [0]:
df1 = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

In [0]:
df1 = df1.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

In [0]:
df1 = df.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

### **Classes - OOPS**

In [0]:
class windows:
    def dense_rank(self, df):
        df_dense_rank = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank
    
    def rank(self, df):
        df_rank = df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_rank
    
    def row_number(self, df):
        df_row_number = df.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_row_number



In [0]:
df_new = df

In [0]:
obj = windows()

df_res = obj.dense_rank(df_new)

### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save(abfss_path("silver", "orders"))

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS databricks_cata.silver.orders_silver
USING DELTA
LOCATION '{abfss_path("silver", "orders")}'
""")